In [1]:
# Loading Data

%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer



# using the SQLite Table to read data.
con = sqlite3.connect('./amazon-fine-food-reviews/database.sqlite') 



#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""", con) 

filtered_data = filtered_data.head(50000)


# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative

In [2]:
filtered_data.shape

(50000, 10)

In [3]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,positive,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,positive,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,positive,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,positive,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,positive,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [4]:
# Data Cleaning:Deduplication

In [5]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [6]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(46072, 10)

In [7]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

92.144

In [8]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]


In [9]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(46071, 10)


positive    38479
negative     7592
Name: Score, dtype: int64

In [10]:
#  Text Preprocessing: Stemming, stop-word removal and Lemmatization.

In [11]:
# find sentences containing HTML tags
import re
i=0;
for sent in final['Text'].values:
    if (len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i += 1;    

        

2
Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [12]:
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))

{'the', 'if', 'my', 'his', 'once', "haven't", 'doesn', 'herself', 'this', "didn't", 'it', 'have', 'who', 'ours', 'hers', 'your', 'but', 'wouldn', "mustn't", 'during', 'm', "doesn't", "should've", 't', 'yourselves', 'we', 'isn', 'after', 'she', 'should', 'down', 'himself', 'needn', 'be', 'too', 'between', 'and', 'on', 'through', 'couldn', 'are', 'further', 'before', "mightn't", 'mightn', 'will', 'over', 'shouldn', 'or', 'there', 'hadn', "don't", 'because', "aren't", 'they', 'these', "needn't", 'doing', 'yourself', 'above', "you'd", 'haven', 'here', "won't", 'only', 'just', 'no', 'was', 'any', "she's", "wasn't", 'other', "wouldn't", 'with', 'i', 's', 'a', "you've", 'that', 'am', 'against', "you'll", "you're", "shouldn't", 'ourselves', 'while', 've', 'y', 'been', 'below', "isn't", 'up', 'don', 'until', 'few', 'hasn', 'why', "hadn't", 'nor', 'their', 'all', 'under', 'then', 'has', 'ain', 'her', 'him', 'o', 'very', 'is', 'me', 'does', 'yours', 'd', 'some', 'by', "couldn't", 'had', 'ma', 'at

In [16]:
#Code for implementing step-by-step the checks mentioned in the pre-processing phase
# this code takes a while to run as it needs to run on 500k sentences.
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [17]:
final['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 

In [18]:
final


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
22620,24750,2734888454,A13ISQV0U9GZIC,Sandikaye,1,1,negative,1192060800,made in china,My dogs loves this chicken but its a product f...,b'dog love chicken product china wont buy anym...
22621,24751,2734888454,A1C298ITT645B6,Hugh G. Pritchard,0,0,positive,1195948800,Dog Lover Delites,Our dogs just love them. I saw them in a pet ...,b'dog love saw pet store tag attach regard mad...
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,positive,1282953600,thirty bucks?,Why is this $[...] when the same product is av...,b'product avail www amazon com victor trap unr...
2547,2775,B00002NCJC,A13RRPGE79XFFH,reader48,0,0,positive,1281052800,Flies Begone,We have used the Victor fly bait for 3 seasons...,b'use victor fli bait season cant beat great p...
1145,1244,B00002Z754,A3B8RCEI0FXFI6,B G Chase,10,10,positive,962236800,WOW Make your own 'slickers' !,I just received my shipment and could hardly w...,b'receiv shipment could hard wait tri product ...
1146,1245,B00002Z754,A29Z5PI9BW2PU3,Robbie,7,7,positive,961718400,Great Product,This was a really good idea and the final prod...,b'realli good idea final product outstand use ...
8696,9527,B00005V3DC,A8KY7S48EW7LW,"A. Daly ""AD""",0,0,positive,1350172800,The Best Cleansing Tea I've Ever Had,I've used this brand for years. If you're feel...,b'ive use brand year your feel clog ate massiv...
8695,9526,B00005V3DC,APASCXWTM041,Ed Raton,0,0,positive,1350604800,"Good, effective product","Good flavor, unique in all the teas that I've ...",b'good flavor uniqu tea ive tri tea effect cle...
8694,9525,B00005V3DC,A2ZYCEEYBUQZND,"Robby ""Robby C""",5,7,positive,1176249600,Best herbal tea for digestion,If you're new to this product you need to be v...,b'your new product need care dosag strong batc...
48736,52952,B00006IDJO,A1NKE5OYRM326V,"The Amazin' Amazon Guy ""Help Me I'm Too Cool!""",2,7,negative,1206835200,They Secretly Switched The Coffee At My Favori...,One of the favorite places that I frequent whe...,b'one favorit place frequent afford nice meal ...


In [19]:
final.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
22620,24750,2734888454,A13ISQV0U9GZIC,Sandikaye,1,1,negative,1192060800,made in china,My dogs loves this chicken but its a product f...,b'dog love chicken product china wont buy anym...
22621,24751,2734888454,A1C298ITT645B6,Hugh G. Pritchard,0,0,positive,1195948800,Dog Lover Delites,Our dogs just love them. I saw them in a pet ...,b'dog love saw pet store tag attach regard mad...
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,positive,1282953600,thirty bucks?,Why is this $[...] when the same product is av...,b'product avail www amazon com victor trap unr...


In [20]:
# store final table into an SQlLite table for future
conn = sqlite3.connect('final.sqlite')
c = conn.cursor()
conn.text_factory = str
final.to_sql("Reviews", conn, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

In [19]:
# Bag of Words(BOW)

In [20]:
# BOW
count_vect = CountVectorizer()
final_counts = count_vect.fit_transform(final['Text'].values)

In [21]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [22]:
final_counts.get_shape()

(4986, 13510)

In [23]:
# Bi-Grams and n-Grams


In [24]:
freq_dist_positive = nltk.FreqDist(all_positive_words)
freq_dist_negative = nltk.FreqDist(all_negative_words)
print("Most Common Positive words : ", freq_dist_positive.most_common(20))
print("Most Common Negative words : ", freq_dist_negative.most_common(20))


Most Common Positive words :  [(b'like', 1812), (b'tast', 1636), (b'good', 1571), (b'flavor', 1549), (b'love', 1468), (b'great', 1442), (b'use', 1269), (b'product', 1204), (b'one', 1193), (b'tri', 1161), (b'coffe', 1027), (b'food', 1017), (b'chip', 997), (b'make', 982), (b'get', 830), (b'tea', 801), (b'bag', 761), (b'buy', 728), (b'best', 710), (b'eat', 709)]
Most Common Negative words :  [(b'like', 444), (b'tast', 432), (b'product', 399), (b'tri', 282), (b'one', 281), (b'flavor', 271), (b'would', 247), (b'food', 241), (b'use', 231), (b'good', 207), (b'buy', 187), (b'order', 185), (b'tea', 182), (b'chip', 180), (b'bag', 179), (b'get', 179), (b'even', 169), (b'make', 162), (b'box', 161), (b'mix', 155)]


In [25]:
# Bi-gram, tri-gram and n-gram
# removing stop words like "not" should be avoided before bulding n-grams
count_vect = CountVectorizer(ngram_range=(1, 2))
final_bigram_counts = count_vect.fit_transform(final['Text'].values)

In [26]:
final_bigram_counts.get_shape()

(4986, 148211)

In [27]:
#TF-IDF

In [28]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1, 2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)

In [29]:
final_tf_idf.get_shape()

(4986, 148211)

In [30]:
features = tf_idf_vect.get_feature_names()
len(features)

148211

In [31]:
features[100:110]

['10 big',
 '10 boxes',
 '10 br',
 '10 broken',
 '10 can',
 '10 cans',
 '10 cents',
 '10 cheaper',
 '10 count',
 '10 cups']

In [32]:
# conert a row in sparsematrix to a numpy array
print(final_tf_idf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [33]:
def top_tfidf_feats(row, features, top_n=25):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [34]:
top_tfidf

,feature,tfidf
0,fly bait,0.274736
1,seasons can,0.274736
2,for seasons,0.274736
3,victor,0.262108
4,bait for,0.262108
5,the victor,0.262108
6,victor fly,0.262108
7,fly,0.246199
8,seasons,0.246199
9,bait,0.240521


In [35]:
#Word2Vect

# Train own Word2Vec model using our text corpus

i = 0
list_of_sent = []
for sent in final['Text'].values:
    filtered_sentences = []
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue
    list_of_sent.append(filtered_sentence)

In [36]:
import gensim

ModuleNotFoundError: No module named 'gensim'

In [ ]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in list_of_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 2
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

/usr/local/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


In [ ]:
# TF-IDF weighted Word2Vec
tfidf_feat = tf_idf_vect.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in list_of_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            # obtain the tf_idfidf of a word in a sentence/review
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
        except:
            pass
    sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1
    

    

In [ ]:
len(tfidf_sent_vectors)
